In [2]:
import pandas as pd
import os
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
#read file from path into a pandas dataframe
path = 'C:/Users/hp/Desktop/rec-flix/data/raw/'
filename = 'netflix_titles.csv'
df = pd.read_csv(os.path.join(path, filename))

In [4]:
df_bow=df[['title','director','cast','listed_in','description']]
print(f'Number of columns with Null values: {df_bow.isna().any().sum()}')

Number of columns with Null values: 2


In [5]:
#drop na values
df_bow.dropna(inplace=True)
print(f'Now, are there Na values: {True if df_bow.isna().any().sum() else False}')

Now, are there Na values: False


C:\Users\hp\AppData\Local\Temp\ipykernel_12464\1070907356.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bow.dropna(inplace=True)


In [6]:
for index,row in df_bow.iterrows():
    description=row['description']
    r=Rake()
    r.extract_keywords_from_text(description)
    key_words_dict_scores=r.get_word_degrees()
    row['Key_words']=list(key_words_dict_scores.keys())

df_bow.drop(columns=['description'],inplace=True)

df_bow['cast'] = df_bow['cast'].map(lambda x : x.split(',')[:3])
df_bow['listed_in'] = df_bow['listed_in'].map(lambda x: x.lower().split(','))
df_bow['director'] = df_bow['director'].map(lambda x: x.split(' '))

for index,row in df_bow.iterrows():
    row['cast']= [x.lower().replace(',',' ') for x in row['cast']]
    row['director']=''.join(row['director']).lower()

df_bow.set_index('title',inplace=True)

df_bow['bag_of_words']=""
columns=df_bow.columns
for index,row in df_bow.iterrows():
    words= ""
    for col in columns:
        if col != 'director':
            words= words + ' '.join(row[col])+ ' '
        else:
            words= words + row[col] + ' '
    row['bag_of_words']=words

df_bow.drop(columns=[col for col in df_bow.columns if col!='bag_of_words'],inplace=True)

C:\Users\hp\AppData\Local\Temp\ipykernel_12464\590408319.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bow.drop(columns=['description'],inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_12464\590408319.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bow['cast'] = df_bow['cast'].map(lambda x : x.split(',')[:3])
C:\Users\hp\AppData\Local\Temp\ipykernel_12464\590408319.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [8]:
count=CountVectorizer()
count_matrix=count.fit_transform(df_bow['bag_of_words'])
indices=pd.Series(df_bow.index)
cosine_sim=cosine_similarity(count_matrix,count_matrix)

In [9]:
def recommendations(Title,cosine_sim=cosine_sim):
    recommended_movies=[]
    idx=indices[indices==Title].index[0]
    score_series=pd.Series(cosine_sim[idx]).sort_values(ascending=False)
    top_10_indexes=list(score_series.iloc[1:11].index)
    
    for i in top_10_indexes:
        recommended_movies.append(list(df_bow.index)[i])
    return recommended_movies

In [18]:
recommended_movies = recommendations('3 Idiots')
print(f"Movies recommended based on your input are :\n {recommended_movies}")

Movies recommended based on your input are :
 ['Talaash', 'PK', 'Gori Tere Pyaar Mein', 'Raja Hindustani', 'Ek Main Aur Ekk Tu', 'Dil Chahta Hai', 'Dil', 'Kapoor & Sons', 'Chup Chup Ke', 'Bewafaa']


In [19]:
recommended_movies = recommendations('Bad Boys')
print(f"Movies recommended based on your input are :\n {recommended_movies}")

Movies recommended based on your input are :
 ['Bad Boys II', 'Blue Streak', 'Wild Wild West', 'Men in Black II', 'Men in Black', 'The Other Guys', "King's Ransom", 'Bright', 'The Super Parental Guardians', 'Talladega Nights: The Ballad of Ricky Bobby']
